# Data Preprocessing

In this notebook, we're demonstrating our data query package implemented in `metalhistory/data_query_api`.
The package takes care of the API calls to LastFM and Musicbrainz form which we will obtain information about the albums we're interested in.


Start with the imports...


In [1]:
import requests
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import metalhistory.data_query_functions as dqf

In [2]:
#Use the pandas extension of tqdm for pretty progress bars
tqdm.pandas()

# Data Preprocessing
Read in a CSV file that has the following structure:

__id__|__artist__|__album__|__MA_Score__|

Wherte 'artist' refers to an artist's or band's name, 'album' refers to a name of an album release, and 'MA_Score' refers to the overall rating on Metal Archives.


In [12]:
df_csv = pd.read_csv('data/HoM.csv')
df_csv = df_csv.dropna(axis=1, how='all').drop('Unnamed: 16', axis=1)
df_csv

,Nr.,Band,Album,Jahr,Monat,Tag,MA
0,1,311,Evolver,2003,7.0,22,NaN
1,2,311,From Chaos,2001,6.0,19,NaN
2,3,311,Mosaic,2017,6.0,23,NaN
3,4,311,Soundsystem,1999,10.0,12,NaN
4,5,1349,Beyond the Apocalypse,2004,4.0,19,9.73
...,...,...,...,...,...,...,...
9355,9356,Zhrine,Unortheta,NaN,NaN,NaN,4.43
9356,9357,Znöwhite,Act of God,NaN,NaN,NaN,8.44
9357,9358,Zonaria,Infamy and the Breed,NaN,NaN,NaN,3.89
9358,9359,Zyklon,World ov Worms,NaN,NaN,NaN,4.03


In [17]:
print(9360/120)

78.0


In [18]:
df_csv.head(9360).tail(120)

,Nr.,Band,Album,Jahr,Monat,Tag,MA
9240,9241,Wolfheart,Wolves of Karelia,NaN,NaN,NaN,3.43
9241,9242,Wolves in the Throne Room,Black Cascade,NaN,NaN,NaN,11.63
9242,9243,Wolves in the Throne Room,Celestial Lineage,NaN,NaN,NaN,7.40
9243,9244,Wolves in the Throne Room,Diadem of 12 Stars,NaN,NaN,NaN,10.67
9244,9245,Wolves in the Throne Room,Malevolent Grain,NaN,NaN,NaN,5.82
...,...,...,...,...,...,...,...
9355,9356,Zhrine,Unortheta,NaN,NaN,NaN,4.43
9356,9357,Znöwhite,Act of God,NaN,NaN,NaN,8.44
9357,9358,Zonaria,Infamy and the Breed,NaN,NaN,NaN,3.89
9358,9359,Zyklon,World ov Worms,NaN,NaN,NaN,4.03


## Extract Album information from LastFM and musicbrainz

Now, we are using the APIs from LastFM and musicbrainz to collect more information about the respective albums.
For demonstration, let's preprocess the albums on position 76 to 100.

*Note: Running it on the full list of 10000 albums would take a lot of time, as the APIs limit an excessive amounts of requests in a short period of time.*

In [29]:
%%time
lastfm = dqf.LastFM()

TAIL = 120
for HEAD in np.arange(0, len(df_csv)+TAIL, TAIL)[1:]:
    start = HEAD-TAIL
    stop = HEAD
    print(start, '-', stop)
    FIELDS = ['artist', 'name', 'release-date', 'listeners', 'playcount', 'tags', 'mbid', 'url', 'image']

    result_df = df_csv.head(HEAD).tail(TAIL)
    result_df['lastfm_info'] = result_df.progress_apply(lambda row: lastfm.get_album_info(artist=row['Band'], album=row['Album'], fields=FIELDS), axis=1)

    result_df = pd.concat([result_df, result_df.lastfm_info.apply(pd.Series)], axis=1)
    result_df = result_df.drop('lastfm_info', axis=1)
    # result_df.rename(columns={'name':'lastfm_album'}, inplace=True)

    start = HEAD-TAIL
    stop = HEAD
    result_df.to_csv('./data/processed/proc_MA_'+ str(start) + '-' + str(HEAD) + '_albums.csv')

0 - 120


  0%|          | 0/120 [00:00<?, ?it/s]

Response code 503. Waiting for 2 seconds.
Response code 503. Waiting for 4 seconds.
Response code 503. Waiting for 4 seconds.
Response code 503. Waiting for 4 seconds.
Response code 503. Waiting for 5 seconds.
Response code 503. Waiting for 4 seconds.
120 - 240


  0%|          | 0/120 [00:00<?, ?it/s]

Response code 503. Waiting for 4 seconds.
Response code 503. Waiting for 4 seconds.
Response code 503. Waiting for 3 seconds.
Response code 503. Waiting for 4 seconds.
Response code 503. Waiting for 3 seconds.
240 - 360


...

Response code 503. Waiting for 3 seconds.
9120 - 9240


  0%|          | 0/120 [00:00<?, ?it/s]

Response code 503. Waiting for 2 seconds.
Response code 503. Waiting for 2 seconds.
9240 - 9360


  0%|          | 0/120 [00:00<?, ?it/s]

CPU times: user 3min 47s, sys: 13.6 s, total: 4min
Wall time: 1h 35min


## Preview the preprocessed data

Let's display some info we gathered about the metal albums placing on positions 76-100 of the top heavy metal albums of history.
We will take a look at the fields:

|__artist__|__album__|__release-date__|__listeners__|__playcount__|__tags__|__ignored tags__|

In [6]:
SHOW_FIELDS =['MA_artist','release-date', 'listeners', 'playcount', 'tags', 'ignored tags']
result_df[SHOW_FIELDS]

,,MA_artist,release-date,listeners,playcount,tags,ignored tags
artist,album,,,,,,
Slayer,Reign in Blood,Slayer,1986-10-07,826535,15664417,"[thrash metal, speed metal, heavy metal]","[albums i own, metal]"
Metallica,Kill 'Em All,Metallica,1983-07-25,583516,14405386,"[thrash metal, heavy metal, speed metal]","[albums i own, metal]"
NaN,NaN,Hades Archer,NaN,NaN,NaN,NaN,NaN
Iron Maiden,Iron Maiden,Iron Maiden,1980-04-14,393721,7110933,[heavy metal],"[albums i own, nwobhm, metal, 1980]"
Metallica,Master of Puppets,Metallica,2003,952876,22537859,"[thrash metal, heavy metal]","[albums i own, metal, favourite albums]"
...,...,...,...,...,...,...,...
NaN,NaN,Ulver,NaN,NaN,NaN,NaN,NaN
Suffocation,Pierced From Within,Suffocation,1995-05-23,53966,992878,"[death metal, technical death metal]","[brutal death metal, 1995, albums i own]"
Black Sabbath,Born Again,Black Sabbath,1983-08-07,84917,1224192,[heavy metal],"[hard rock, 1983, classic rock, albums i own]"


## Export
We can now export the preprocessed dataframe to a csv file.

In [6]:
start = HEAD-TAIL
stop = HEAD
result_df.sort_values(by='listeners', ascending=False).to_csv('./data/proc_MA_'+ str(start) + '-' + str(HEAD) + '_albums.csv')

# Next Steps
Now, that we have exported our data to a csv file, we can use our visualization functions to further explore the history of heavy metal. Follow the link below:


[1-visualizations.ipynb](1-visualizations.ipynb)